In [1]:
#imports

from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
import numpy as np

In [2]:
#this block of code extracts team names that will be used for urls

teams_url = 'http://www.footballdb.com/teams/index.html'

with urllib.request.urlopen(teams_url) as url:
    teams_page = url.read()

In [3]:
#preview of the html

teams_soup = BeautifulSoup(teams_page, 'html.parser')
print(teams_soup.prettify()[0:1000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="initial-scale=1.0, width=device-width" name="viewport"/>
  <title>
   Teams | The Football Database
  </title>
  <meta content="View stats and statistics, rosters, results, draft results and transactions for current and defunct NFL teams" name="description"/>
  <meta content="index,follow" name="robots"/>
  <meta content="FootballDB.com" property="og:site_name"/>
  <meta content="Teams | The Football Database" property="og:title">
   <meta content="website" property="og:type"/>
   <meta content="View stats and statistics, rosters, results, draft results and transactions for current and defunct NFL teams" property="og:description"/>
   <meta content="https://www.footballdb.com/images/footballdb_200x200.png" property="og:image"/>
   <meta content="https://www.footballdb.com/teams/index.html" property="og:url"/>
   <meta content="summary" name="twitt

In [4]:
#list with html tags

teams_list = teams_soup.find_all('a', attrs = {'href': re.compile('/teams/nfl/')})[0:31]
teams_list

[<a href="/teams/nfl/buffalo-bills">Buffalo Bills</a>,
 <a href="/teams/nfl/miami-dolphins">Miami Dolphins</a>,
 <a href="/teams/nfl/new-england-patriots">New England Patriots</a>,
 <a href="/teams/nfl/new-york-jets">New York Jets</a>,
 <a href="/teams/nfl/baltimore-ravens">Baltimore Ravens</a>,
 <a href="/teams/nfl/cincinnati-bengals">Cincinnati Bengals</a>,
 <a href="/teams/nfl/cleveland-browns">Cleveland Browns</a>,
 <a href="/teams/nfl/pittsburgh-steelers">Pittsburgh Steelers</a>,
 <a href="/teams/nfl/houston-texans">Houston Texans</a>,
 <a href="/teams/nfl/indianapolis-colts">Indianapolis Colts</a>,
 <a href="/teams/nfl/jacksonville-jaguars">Jacksonville Jaguars</a>,
 <a href="/teams/nfl/tennessee-titans">Tennessee Titans</a>,
 <a href="/teams/nfl/denver-broncos">Denver Broncos</a>,
 <a href="/teams/nfl/kansas-city-chiefs">Kansas City Chiefs</a>,
 <a href="/teams/nfl/los-angeles-chargers">Los Angeles Chargers</a>,
 <a href="/teams/nfl/oakland-raiders">Oakland Raiders</a>,
 <a href

In [5]:
#list wihtout html

teams_list_clean = []

for i in range(0, len(teams_list)):
    teams_list_clean.append(re.sub('<[^<]+?>', '', str(teams_list[i])).lower().replace(' ', '-'))
    
teams_list_clean

['buffalo-bills',
 'miami-dolphins',
 'new-england-patriots',
 'new-york-jets',
 'baltimore-ravens',
 'cincinnati-bengals',
 'cleveland-browns',
 'pittsburgh-steelers',
 'houston-texans',
 'indianapolis-colts',
 'jacksonville-jaguars',
 'tennessee-titans',
 'denver-broncos',
 'kansas-city-chiefs',
 'los-angeles-chargers',
 'oakland-raiders',
 'dallas-cowboys',
 'new-york-giants',
 'philadelphia-eagles',
 'washington-redskins',
 'chicago-bears',
 'detroit-lions',
 'green-bay-packers',
 'minnesota-vikings',
 'atlanta-falcons',
 'carolina-panthers',
 'new-orleans-saints',
 'tampa-bay-buccaneers',
 'arizona-cardinals',
 'los-angeles-rams',
 'san-francisco-49ers']

In [6]:
#instantiate dataframes. the columns are what is available on the website of interest

passing_cols = ['player', 'pass_attempts', 'completions', 'compl_percent', 'passing_yards', 'yards_per_pass', 'passing_touchdowns', 'touchdown_percentage', 'interceptions', 'int_percentage', 'long', 'sack', 'loss', 'passer_rating']
rushing_cols = ['player', 'games_played', 'rush_attempts', 'rush_yards', 'yards_per_rush', 'yards_per_game', 'long', 'rush_touchdowns', 'first_downs']
receiving_cols = ['player', 'games', 'receptions', 'receiving_yards', 'yards_per_catch', 'yards_per_game', 'long', 'receiving_touchdowns', 'first_downs', 'targets', 'yards_after_catch']
kick_returns_cols = ['player', 'kick_returns', 'kick_return_yards', 'kick_return_average', 'fair_catches', 'long', 'touchdowns']
punt_returns_cols = ['player', 'punt_returns', 'punt_return_yards', 'punt_return_average', 'fair_catches', 'long', 'touchdowns']
punts_cols = ['player', 'punts', 'yards', 'average', 'long', 'touchbacks', 'inside_twenty', 'out_of_bounds', 'fair_catches', 'downed', 'blocked', 'net', 'return', 'return_yards', 'touchdowns']
kicking_cols = ['player', 'pat', 'field_goals', '0_19', '20_29', '30-39', '40-49', '50_plus', 'long', 'points']
kickoffs_cols = ['player', 'number', 'yards', 'average', 'long', 'touchbacks', 'out_of_bounds', 'returns', 'return_yards', 'touchdowns', 'osk', 'osr']
defense_cols = ['player', 'interceptions', 'yards', 'average', 'long', 'touchdowns', 'solo_tackles', 'assists',  'total_tackles', 'sacks', 'yards_for_loss']

passing_df = pd.DataFrame(columns = passing_cols)
rushing_df = pd.DataFrame(columns = rushing_cols)
receiving_df = pd.DataFrame(columns = receiving_cols)
kick_returns_df = pd.DataFrame(columns = kick_returns_cols)
punt_returns_df = pd.DataFrame(columns = punt_returns_cols)
punts_df = pd.DataFrame(columns = punts_cols)
kicking_df = pd.DataFrame(columns = kicking_cols)
kickoffs_df = pd.DataFrame(columns = kickoffs_cols)
defense_df = pd.DataFrame(columns = defense_cols)

In [18]:
#all stats

years = range(2000, 2017)

for i in range(0, len(teams_list_clean)):
    for y in range(2000, 2018):
        url = 'http://www.footballdb.com/teams/nfl/' + teams_list_clean[i] + '/stats/' + str(y)
        try: #account for newer expansion teams
            with urllib.request.urlopen(url) as url:
                page = url.read()
                soup = BeautifulSoup(page, 'html.parser')
                all = soup.find_all('td')
        except:
            continue

        #passing

        passing_html = soup.find('div', attrs = {'class': 'divToggle_offense', 'id': 'divToggle_P'})
        passing_players = str(passing_html).count('/players/') // 2
        passing_count = int(14 * ((str(passing_html).count('/players/') // 2) + 2))

        passing = all[0:passing_count]
        passing_clean = []

        for x in range(0, len(passing)):
            passing_clean.append(re.sub('<[^<]+?>', '', str(passing[x])))

        passing_df2 = pd.DataFrame(np.array(passing_clean).reshape(passing_players + 2, len(passing_cols)), columns = passing_cols)
        passing_df2['team'] = teams_list_clean[i]
        passing_df = passing_df.append(passing_df2)
        passing_df['year'] = y

        print('successfully imported passing stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #rushing

        rushing_html = soup.find('div', attrs = {'class': 'divToggle_offense', 'id': 'divToggle_R'})
        rushing_players = str(rushing_html).count('/players/') // 2
        rushing_count = int(9 * ((str(rushing_html).count('/players/') // 2) + 2))

        rushing = all[passing_count:rushing_count + passing_count] #update this
        rushing_clean = []

        for x in range(0, len(rushing)):
            rushing_clean.append(re.sub('<[^<]+?>', '', str(rushing[x])))

        rushing_df2 = pd.DataFrame(np.array(rushing_clean).reshape(rushing_players + 2, len(rushing_cols)), columns = rushing_cols)
        rushing_df2['team'] = teams_list_clean[i]
        rushing_df = rushing_df.append(rushing_df2)
        rushing_df['year'] = y

        print('successfully imported rushing stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #receiving

        receiving_html = soup.find('div', attrs = {'class': 'divToggle_offense', 'id': 'divToggle_C'})
        receiving_players = str(receiving_html).count('/players/') // 2
        receiving_count = int(11 * ((str(receiving_html).count('/players/') // 2) + 2))

        receiving = all[passing_count + rushing_count:receiving_count + rushing_count + passing_count]
        receiving_clean = []

        for x in range(0, len(receiving)):
            receiving_clean.append(re.sub('<[^<]+?>', '', str(receiving[x])))

        receiving_df2 = pd.DataFrame(np.array(receiving_clean).reshape(receiving_players + 2, len(receiving_cols)), columns = receiving_cols)
        receiving_df2['team'] = teams_list_clean[i]
        receiving_df = receiving_df.append(receiving_df2)
        receiving_df['year'] = y

        print('successfully imported receiving stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #kick returns

        kick_returns_html = soup.find('div', attrs = {'class': 'divToggle_special hidden-xs', 'id': 'divToggle_KR'})
        kick_returns_players = str(kick_returns_html).count('/players/') // 2
        kick_returns_count = int(7 * ((str(kick_returns_html).count('/players/') // 2) + 2))

        kick_returns = all[passing_count + rushing_count + receiving_count:kick_returns_count + receiving_count + rushing_count + passing_count]
        kick_returns_clean = []

        for x in range(0, len(kick_returns)):
            kick_returns_clean.append(re.sub('<[^<]+?>', '', str(kick_returns[x])))

        kick_returns_df2 = pd.DataFrame(np.array(kick_returns_clean).reshape(kick_returns_players + 2, len(kick_returns_cols)), columns = kick_returns_cols)
        kick_returns_df2['team'] = teams_list_clean[i]
        kick_returns_df = kick_returns_df.append(kick_returns_df2)
        kick_returns_df['year'] = y

        print('successfully imported kick return stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #punt returns

        punt_returns_html = soup.find('div', attrs = {'class': 'divToggle_special hidden-xs', 'id': 'divToggle_PR'})
        punt_returns_players = str(punt_returns_html).count('/players/') // 2
        punt_returns_count = int(7 * ((str(punt_returns_html).count('/players/') // 2) + 2))

        punt_returns = all[passing_count + rushing_count + receiving_count + kick_returns_count:punt_returns_count + kick_returns_count + receiving_count + rushing_count + passing_count]
        punt_returns_clean = []

        for x in range(0, len(punt_returns)):
            punt_returns_clean.append(re.sub('<[^<]+?>', '', str(punt_returns[x])))

        punt_returns_df2 = pd.DataFrame(np.array(punt_returns_clean).reshape(punt_returns_players + 2, len(punt_returns_cols)), columns = punt_returns_cols)
        punt_returns_df2['team'] = teams_list_clean[i]
        punt_returns_df = punt_returns_df.append(punt_returns_df2)
        punt_returns_df['year'] = y

        print('successfully imported punt return stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #punting

        punts_html = soup.find('div', attrs = {'class': 'divToggle_special hidden-xs', 'id': 'divToggle_PR'})
        punts_players = str(punt_returns_html).count('/players/') // 2
        punts_count = int(15 * ((str(punt_returns_html).count('/players/') // 2) + 2))

        punts = all[passing_count + rushing_count + receiving_count + kick_returns_count + punt_returns_count:punts_count + punt_returns_count + kick_returns_count + receiving_count + rushing_count + passing_count]
        punts_clean = []

        for x in range(0, len(punts)):
            punts_clean.append(re.sub('<[^<]+?>', '', str(punts[x])))

        punts_df2 = pd.DataFrame(np.array(punts_clean).reshape(punts_players + 2, len(punts_cols)), columns = punts_cols)
        punts_df2['team'] = teams_list_clean[i]
        punts_df = punts_df.append(punts_df2)
        punts_df['year'] = y

        print('successfully imported punting stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #kicking

        kicking_html = soup.find('div', attrs = {'class': 'divToggle_special hidden-xs', 'id': 'divToggle_U'})
        kicking_players = str(kicking_html).count('/players/') // 2
        kicking_count = int(10 * ((str(kicking_html).count('/players/') // 2) + 2))

        kicking = all[passing_count + rushing_count + receiving_count + kick_returns_count + punt_returns_count + punts_count:kicking_count + punts_count + punt_returns_count + kick_returns_count + receiving_count + rushing_count + passing_count]
        kicking_clean = []

        for x in range(0, len(kicking)):
            kicking_clean.append(re.sub('<[^<]+?>', '', str(kicking[x])))

        kicking_df2 = pd.DataFrame(np.array(kicking_clean).reshape(kicking_players + 2, len(kicking_cols)), columns = kicking_cols)
        kicking_df2['team'] = teams_list_clean[i]
        kicking_df = kicking_df.append(kicking_df2)
        kicking_df['year'] = y

        print('successfully imported kicking stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #kickoffs

        kickoffs_html = soup.find('div', attrs = {'class': 'divToggle_special hidden-xs', 'id': 'divToggle_O'})
        kickoffs_players = str(kickoffs_html).count('/players/') // 2
        kickoffs_count = int(12 * ((str(kickoffs_html).count('/players/') // 2) + 2))

        kickoffs = all[passing_count + rushing_count + receiving_count + kick_returns_count + punt_returns_count + punts_count + kicking_count:kickoffs_count + kicking_count + punts_count + punt_returns_count + kick_returns_count + receiving_count + rushing_count + passing_count]
        kickoffs_clean = []

        for x in range(0, len(kickoffs)):
            kickoffs_clean.append(re.sub('<[^<]+?>', '', str(kickoffs[x])))

        kickoffs_df2 = pd.DataFrame(np.array(kickoffs_clean).reshape(kickoffs_players + 2, len(kickoffs_cols)), columns = kickoffs_cols)
        kickoffs_df2['team'] = teams_list_clean[i]
        kickoffs_df = kickoffs_df.append(kickoffs_df2)
        kickoffs_df['year'] = y

        print('successfully imported kickoff stats for the ' + teams_list_clean[i] + ' ' + str(y))

        #defense

        defense_html = soup.find('div', attrs = {'class': 'divToggle_defense hidden-xs', 'id': 'divToggle_D'})
        defense_players = str(defense_html).count('/players/') // 2
        defense_count = int(11 * ((str(defense_html).count('/players/') // 2) + 2))

        defense = all[passing_count + rushing_count + receiving_count + kick_returns_count + punt_returns_count + punts_count + kicking_count + kickoffs_count:defense_count + kickoffs_count + kicking_count + punts_count + punt_returns_count + kick_returns_count + receiving_count + rushing_count + passing_count]
        defense_clean = []

        for x in range(0, len(defense)):
            defense_clean.append(re.sub('<[^<]+?>', '', str(defense[x])))

        defense_df2 = pd.DataFrame(np.array(defense_clean).reshape(defense_players + 2, len(defense_cols)), columns = defense_cols)
        defense_df2['team'] = teams_list_clean[i]
        defense_df = defense_df.append(defense_df2)
        defense_df['year'] = y

        print('successfully imported defense stats for the ' + teams_list_clean[i] + ' ' + str(y))

successfully imported passing stats for the buffalo-bills 2000
successfully imported rushing stats for the buffalo-bills 2000
successfully imported receiving stats for the buffalo-bills 2000
successfully imported kick return stats for the buffalo-bills 2000
successfully imported punt return stats for the buffalo-bills 2000
successfully imported punting stats for the buffalo-bills 2000
successfully imported kicking stats for the buffalo-bills 2000
successfully imported kickoff stats for the buffalo-bills 2000
successfully imported defense stats for the buffalo-bills 2000
successfully imported passing stats for the buffalo-bills 2001
successfully imported rushing stats for the buffalo-bills 2001
successfully imported receiving stats for the buffalo-bills 2001
successfully imported kick return stats for the buffalo-bills 2001
successfully imported punt return stats for the buffalo-bills 2001
successfully imported punting stats for the buffalo-bills 2001
successfully imported kicking stats

successfully imported passing stats for the buffalo-bills 2015
successfully imported rushing stats for the buffalo-bills 2015
successfully imported receiving stats for the buffalo-bills 2015
successfully imported kick return stats for the buffalo-bills 2015
successfully imported punt return stats for the buffalo-bills 2015
successfully imported punting stats for the buffalo-bills 2015
successfully imported kicking stats for the buffalo-bills 2015
successfully imported kickoff stats for the buffalo-bills 2015
successfully imported defense stats for the buffalo-bills 2015
successfully imported passing stats for the buffalo-bills 2016
successfully imported rushing stats for the buffalo-bills 2016
successfully imported receiving stats for the buffalo-bills 2016
successfully imported kick return stats for the buffalo-bills 2016
successfully imported punt return stats for the buffalo-bills 2016
successfully imported punting stats for the buffalo-bills 2016
successfully imported kicking stats

successfully imported passing stats for the miami-dolphins 2012
successfully imported rushing stats for the miami-dolphins 2012
successfully imported receiving stats for the miami-dolphins 2012
successfully imported kick return stats for the miami-dolphins 2012
successfully imported punt return stats for the miami-dolphins 2012
successfully imported punting stats for the miami-dolphins 2012
successfully imported kicking stats for the miami-dolphins 2012
successfully imported kickoff stats for the miami-dolphins 2012
successfully imported defense stats for the miami-dolphins 2012
successfully imported passing stats for the miami-dolphins 2013
successfully imported rushing stats for the miami-dolphins 2013
successfully imported receiving stats for the miami-dolphins 2013
successfully imported kick return stats for the miami-dolphins 2013
successfully imported punt return stats for the miami-dolphins 2013
successfully imported punting stats for the miami-dolphins 2013
successfully importe

successfully imported passing stats for the new-england-patriots 2008
successfully imported rushing stats for the new-england-patriots 2008
successfully imported receiving stats for the new-england-patriots 2008
successfully imported kick return stats for the new-england-patriots 2008
successfully imported punt return stats for the new-england-patriots 2008
successfully imported punting stats for the new-england-patriots 2008
successfully imported kicking stats for the new-england-patriots 2008
successfully imported kickoff stats for the new-england-patriots 2008
successfully imported defense stats for the new-england-patriots 2008
successfully imported passing stats for the new-england-patriots 2009
successfully imported rushing stats for the new-england-patriots 2009
successfully imported receiving stats for the new-england-patriots 2009
successfully imported kick return stats for the new-england-patriots 2009
successfully imported punt return stats for the new-england-patriots 2009


successfully imported passing stats for the new-york-jets 2004
successfully imported rushing stats for the new-york-jets 2004
successfully imported receiving stats for the new-york-jets 2004
successfully imported kick return stats for the new-york-jets 2004
successfully imported punt return stats for the new-york-jets 2004
successfully imported punting stats for the new-york-jets 2004
successfully imported kicking stats for the new-york-jets 2004
successfully imported kickoff stats for the new-york-jets 2004
successfully imported defense stats for the new-york-jets 2004
successfully imported passing stats for the new-york-jets 2005
successfully imported rushing stats for the new-york-jets 2005
successfully imported receiving stats for the new-york-jets 2005
successfully imported kick return stats for the new-york-jets 2005
successfully imported punt return stats for the new-york-jets 2005
successfully imported punting stats for the new-york-jets 2005
successfully imported kicking stats

...

successfully imported passing stats for the new-orleans-saints 2014
successfully imported rushing stats for the new-orleans-saints 2014
successfully imported receiving stats for the new-orleans-saints 2014
successfully imported kick return stats for the new-orleans-saints 2014
successfully imported punt return stats for the new-orleans-saints 2014
successfully imported punting stats for the new-orleans-saints 2014
successfully imported kicking stats for the new-orleans-saints 2014
successfully imported kickoff stats for the new-orleans-saints 2014
successfully imported defense stats for the new-orleans-saints 2014
successfully imported passing stats for the new-orleans-saints 2015
successfully imported rushing stats for the new-orleans-saints 2015
successfully imported receiving stats for the new-orleans-saints 2015
successfully imported kick return stats for the new-orleans-saints 2015
successfully imported punt return stats for the new-orleans-saints 2015
successfully imported puntin

successfully imported passing stats for the tampa-bay-buccaneers 2009
successfully imported rushing stats for the tampa-bay-buccaneers 2009
successfully imported receiving stats for the tampa-bay-buccaneers 2009
successfully imported kick return stats for the tampa-bay-buccaneers 2009
successfully imported punt return stats for the tampa-bay-buccaneers 2009
successfully imported punting stats for the tampa-bay-buccaneers 2009
successfully imported kicking stats for the tampa-bay-buccaneers 2009
successfully imported kickoff stats for the tampa-bay-buccaneers 2009
successfully imported defense stats for the tampa-bay-buccaneers 2009
successfully imported passing stats for the tampa-bay-buccaneers 2010
successfully imported rushing stats for the tampa-bay-buccaneers 2010
successfully imported receiving stats for the tampa-bay-buccaneers 2010
successfully imported kick return stats for the tampa-bay-buccaneers 2010
successfully imported punt return stats for the tampa-bay-buccaneers 2010


successfully imported passing stats for the arizona-cardinals 2004
successfully imported rushing stats for the arizona-cardinals 2004
successfully imported receiving stats for the arizona-cardinals 2004
successfully imported kick return stats for the arizona-cardinals 2004
successfully imported punt return stats for the arizona-cardinals 2004
successfully imported punting stats for the arizona-cardinals 2004
successfully imported kicking stats for the arizona-cardinals 2004
successfully imported kickoff stats for the arizona-cardinals 2004
successfully imported defense stats for the arizona-cardinals 2004
successfully imported passing stats for the arizona-cardinals 2005
successfully imported rushing stats for the arizona-cardinals 2005
successfully imported receiving stats for the arizona-cardinals 2005
successfully imported kick return stats for the arizona-cardinals 2005
successfully imported punt return stats for the arizona-cardinals 2005
successfully imported punting stats for th

successfully imported passing stats for the los-angeles-rams 2016
successfully imported rushing stats for the los-angeles-rams 2016
successfully imported receiving stats for the los-angeles-rams 2016
successfully imported kick return stats for the los-angeles-rams 2016
successfully imported punt return stats for the los-angeles-rams 2016
successfully imported punting stats for the los-angeles-rams 2016
successfully imported kicking stats for the los-angeles-rams 2016
successfully imported kickoff stats for the los-angeles-rams 2016
successfully imported defense stats for the los-angeles-rams 2016
successfully imported passing stats for the los-angeles-rams 2017
successfully imported rushing stats for the los-angeles-rams 2017
successfully imported receiving stats for the los-angeles-rams 2017
successfully imported kick return stats for the los-angeles-rams 2017
successfully imported punt return stats for the los-angeles-rams 2017
successfully imported punting stats for the los-angeles-

successfully imported passing stats for the san-francisco-49ers 2012
successfully imported rushing stats for the san-francisco-49ers 2012
successfully imported receiving stats for the san-francisco-49ers 2012
successfully imported kick return stats for the san-francisco-49ers 2012
successfully imported punt return stats for the san-francisco-49ers 2012
successfully imported punting stats for the san-francisco-49ers 2012
successfully imported kicking stats for the san-francisco-49ers 2012
successfully imported kickoff stats for the san-francisco-49ers 2012
successfully imported defense stats for the san-francisco-49ers 2012
successfully imported passing stats for the san-francisco-49ers 2013
successfully imported rushing stats for the san-francisco-49ers 2013
successfully imported receiving stats for the san-francisco-49ers 2013
successfully imported kick return stats for the san-francisco-49ers 2013
successfully imported punt return stats for the san-francisco-49ers 2013
successfully i

In [19]:
#output to csv files

passing_df.to_csv('nfl_passing.csv')
rushing_df.to_csv('nfl_rushing.csv')
receiving_df.to_csv('nfl_receiving.csv')
kick_returns_df.to_csv('nfl_kick_returns.csv')
punt_returns_df.to_csv('nfl_punt_returns.csv')
kicking_df.to_csv('nfl_kicking.csv')
kickoffs_df.to_csv('nfl_kickoff.csv')
defense_df.to_csv('nfl_defense.csv')

In [9]:
#all code below was used for testing (more output to understand what is going on)

url = 'http://www.footballdb.com/teams/nfl/san-francisco-49ers/stats'

with urllib.request.urlopen(url) as url:
    teams_page = url.read()

In [19]:
soup = BeautifulSoup(page, 'html.parser')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="initial-scale=1.0, width=device-width" name="viewport"/>
  <title>
   San Francisco 49ers Statistics | The Football Database
  </title>
  <meta content="San Francisco 49ers stats and statistics for the 2017 NFL season, including rushing, passing, receiving, kickoff returns, punt returns, punting, kicking and defense" name="description"/>
  <meta content="index,follow" name="robots"/>
  <meta content="FootballDB.com" property="og:site_name"/>
  <meta content="San Francisco 49ers Statistics | The Football Database" property="og:title">
   <meta content="website" property="og:type"/>
   <meta content="San Francisco 49ers stats and statistics for the 2017 NFL season, including rushing, passing, receiving, kickoff returns, punt returns, punting, kicking and defense" property="og:description"/>
   <meta content="https://www.footballdb.com/images/footbal

In [20]:
passing_html = soup.find('div', attrs = {'class': 'divToggle_offense', 'id': 'divToggle_P'})
passing_html

<div class="divToggle_offense" id="divToggle_P">
<a name="p"></a>
<h2>Passing</h2>
<table class="statistics scrollable">
<thead>
<tr class="header right">
<th class="left" width="120">Player</th>
<th>Att</th>
<th>Cmp</th>
<th>Pct</th>
<th>Yds</th>
<th>YPA</th>
<th>TD</th>
<th>TD%</th>
<th>Int</th>
<th>Int%</th>
<th>Lg</th>
<th>Sack</th>
<th>Loss</th>
<th>Rate</th>
</tr>
</thead>
<tbody>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/jimmy-garoppolo-garopji01" title="Jimmy Garoppolo Stats">Jimmy Garoppolo</a></span><span class="visible-xs"><a href="/players/jimmy-garoppolo-garopji01" title="Jimmy Garoppolo Stats">J. Garoppolo</a></span></td><td>178</td><td>120</td><td>67.4</td><td>1,560</td><td>8.8</td><td>7</td><td>3.9</td><td>5</td><td>2.8</td><td>61</td><td>8</td><td>57</td><td>96.2</td></tr>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/cj-beathard-beathcj01" title="C.J. Beathard Stats">C.J. Beathard</a></span>

In [10]:
#calculate number of players and total values

passing_players = str(passing_html).count('/players/') // 2
passing_count = int(14 * ((str(passing_html).count('/players/') // 2) + 2))
passing_count

70

In [11]:
print(type(passing_count))

<class 'int'>


In [12]:
all = soup.find_all('td')
passing = all[0:passing_count]
passing

[<td class="left"><span class="hidden-xs"><a href="/players/jimmy-garoppolo-garopji01" title="Jimmy Garoppolo Stats">Jimmy Garoppolo</a></span><span class="visible-xs"><a href="/players/jimmy-garoppolo-garopji01" title="Jimmy Garoppolo Stats">J. Garoppolo</a></span></td>,
 <td>178</td>,
 <td>120</td>,
 <td>67.4</td>,
 <td>1,560</td>,
 <td>8.8</td>,
 <td>7</td>,
 <td>3.9</td>,
 <td>5</td>,
 <td>2.8</td>,
 <td>61</td>,
 <td>8</td>,
 <td>57</td>,
 <td>96.2</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/cj-beathard-beathcj01" title="C.J. Beathard Stats">C.J. Beathard</a></span><span class="visible-xs"><a href="/players/cj-beathard-beathcj01" title="C.J. Beathard Stats">C. Beathard</a></span></td>,
 <td>224</td>,
 <td>123</td>,
 <td>54.9</td>,
 <td>1,430</td>,
 <td>6.4</td>,
 <td>4</td>,
 <td>1.8</td>,
 <td>6</td>,
 <td>2.7</td>,
 <td>83t</td>,
 <td>19</td>,
 <td>141</td>,
 <td>69.2</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/brian-hoyer-hoyerbr01" t

In [13]:
rushing_html = soup.find('div', attrs = {'class': 'divToggle_offense', 'id': 'divToggle_R'})
rushing_html

<div class="divToggle_offense" id="divToggle_R">
<a name="r"></a>
<h2>Rushing</h2>
<table class="statistics scrollable">
<thead>
<tr class="header right"><th class="left" width="120">Player</th><th>Gms</th><th>Att</th><th>Yds</th><th>Avg</th><th>YPG</th><th>Lg</th><th>TD</th><th>FD</th></tr>
</thead>
<tbody>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">Carlos Hyde</a></span><span class="visible-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">C. Hyde</a></span></td><td>16</td><td>240</td><td>938</td><td>3.91</td><td>58.6</td><td>61</td><td>8</td><td>44</td></tr>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/matt-breida-breidma01" title="Matt Breida Stats">Matt Breida</a></span><span class="visible-xs"><a href="/players/matt-breida-breidma01" title="Matt Breida Stats">M. Breida</a></span></td><td>16</td><td>105</td><td>465</td><td>4.43</td><td>2

In [14]:
rushing_players = str(rushing_html).count('/players/') // 2
rushing_count = int(9 * ((str(rushing_html).count('/players/') // 2) + 2))
rushing_count

90

In [15]:
rushing = all[passing_count:rushing_count + passing_count]
rushing

[<td class="left"><span class="hidden-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">Carlos Hyde</a></span><span class="visible-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">C. Hyde</a></span></td>,
 <td>16</td>,
 <td>240</td>,
 <td>938</td>,
 <td>3.91</td>,
 <td>58.6</td>,
 <td>61</td>,
 <td>8</td>,
 <td>44</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/matt-breida-breidma01" title="Matt Breida Stats">Matt Breida</a></span><span class="visible-xs"><a href="/players/matt-breida-breidma01" title="Matt Breida Stats">M. Breida</a></span></td>,
 <td>16</td>,
 <td>105</td>,
 <td>465</td>,
 <td>4.43</td>,
 <td>29.1</td>,
 <td>33t</td>,
 <td>2</td>,
 <td>22</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/cj-beathard-beathcj01" title="C.J. Beathard Stats">C.J. Beathard</a></span><span class="visible-xs"><a href="/players/cj-beathard-beathcj01" title="C.J. Beathard Stats">C. Beathard</a></span></td>,
 <td>7</td>

In [16]:
receiving_html = soup.find('div', attrs = {'class': 'divToggle_offense', 'id': 'divToggle_C'})
receiving_html

<div class="divToggle_offense" id="divToggle_C">
<a name="c"></a>
<h2>Receiving</h2>
<table class="statistics scrollable">
<thead>
<tr class="header right"><th class="left" width="120">Player</th><th>Gms</th><th>Rec</th><th>Yds</th><th>Avg</th><th>YPG</th><th>Lg</th><th>TD</th><th>FD</th><th>Tar</th><th>YAC</th></tr>
</thead>
<tbody>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">Carlos Hyde</a></span><span class="visible-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">C. Hyde</a></span></td><td>16</td><td>59</td><td>350</td><td>5.93</td><td>21.9</td><td>18</td><td>0</td><td>17</td><td>88</td><td>322</td></tr>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/marquise-goodwin-goodwma02" title="Marquise Goodwin Stats">Marquise Goodwin</a></span><span class="visible-xs"><a href="/players/marquise-goodwin-goodwma02" title="Marquise Goodwin Stats">M. Go

In [17]:
receiving_players = str(receiving_html).count('/players/') // 2
receiving_count = int(11 * ((str(receiving_html).count('/players/') // 2) + 2))
receiving_count

176

In [18]:
receiving = all[passing_count + rushing_count:receiving_count + rushing_count + passing_count]
receiving

[<td class="left"><span class="hidden-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">Carlos Hyde</a></span><span class="visible-xs"><a href="/players/carlos-hyde-hydeca01" title="Carlos Hyde Stats">C. Hyde</a></span></td>,
 <td>16</td>,
 <td>59</td>,
 <td>350</td>,
 <td>5.93</td>,
 <td>21.9</td>,
 <td>18</td>,
 <td>0</td>,
 <td>17</td>,
 <td>88</td>,
 <td>322</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/marquise-goodwin-goodwma02" title="Marquise Goodwin Stats">Marquise Goodwin</a></span><span class="visible-xs"><a href="/players/marquise-goodwin-goodwma02" title="Marquise Goodwin Stats">M. Goodwin</a></span></td>,
 <td>16</td>,
 <td>56</td>,
 <td>962</td>,
 <td>17.18</td>,
 <td>60.1</td>,
 <td>83t</td>,
 <td>2</td>,
 <td>46</td>,
 <td>105</td>,
 <td>198</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/trent-taylor-taylotr04" title="Trent Taylor Stats">Trent Taylor</a></span><span class="visible-xs"><a href="/players/trent-ta

In [19]:
kick_returns_html = soup.find('div', attrs = {'class': 'divToggle_special hidden-xs', 'id': 'divToggle_KR'})
kick_returns_html

<div class="divToggle_special hidden-xs" id="divToggle_KR">
<a name="kr"></a>
<h2>Kickoff Returns</h2>
<table class="statistics scrollable">
<thead>
<tr class="header right"><th class="left" width="120">Player</th><th>Num</th><th>Yds</th><th>Avg</th><th>FC</th><th>Lg</th><th>TD</th></tr>
</thead>
<tbody>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">Victor Bolden</a></span><span class="visible-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">V. Bolden</a></span></td><td>19</td><td>396</td><td>20.84</td><td>0</td><td>34</td><td>0</td></tr>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/raheem-mostert-mostera01" title="Raheem Mostert Stats">Raheem Mostert</a></span><span class="visible-xs"><a href="/players/raheem-mostert-mostera01" title="Raheem Mostert Stats">R. Mostert</a></span></td><td>5</td><td>83</td><td>16.60</td><td>0</td><td>21<

In [20]:
kick_returns_players = str(kick_returns_html).count('/players/') // 2
kick_returns_count = int(7 * ((str(kick_returns_html).count('/players/') // 2) + 2))
kick_returns_count

49

In [21]:
kick_returns = all[passing_count + rushing_count + receiving_count:kick_returns_count + receiving_count + rushing_count + passing_count]
kick_returns

[<td class="left"><span class="hidden-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">Victor Bolden</a></span><span class="visible-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">V. Bolden</a></span></td>,
 <td>19</td>,
 <td>396</td>,
 <td>20.84</td>,
 <td>0</td>,
 <td>34</td>,
 <td>0</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/raheem-mostert-mostera01" title="Raheem Mostert Stats">Raheem Mostert</a></span><span class="visible-xs"><a href="/players/raheem-mostert-mostera01" title="Raheem Mostert Stats">R. Mostert</a></span></td>,
 <td>5</td>,
 <td>83</td>,
 <td>16.60</td>,
 <td>0</td>,
 <td>21</td>,
 <td>0</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/matt-breida-breidma01" title="Matt Breida Stats">Matt Breida</a></span><span class="visible-xs"><a href="/players/matt-breida-breidma01" title="Matt Breida Stats">M. Breida</a></span></td>,
 <td>5</td>,
 <td>83</td>,
 <td>16.60</td>,
 <td>0</td

In [22]:
punt_returns_html = soup.find('div', attrs = {'class': 'divToggle_special hidden-xs', 'id': 'divToggle_PR'})
punt_returns_html

<div class="divToggle_special hidden-xs" id="divToggle_PR">
<a name="pr"></a>
<h2>Punt Returns</h2>
<table class="statistics scrollable">
<thead>
<tr class="header right"><th class="left" width="120">Player</th><th>Num</th><th>Yds</th><th>Avg</th><th>FC</th><th>Lg</th><th>TD</th></tr>
</thead>
<tbody>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/trent-taylor-taylotr04" title="Trent Taylor Stats">Trent Taylor</a></span><span class="visible-xs"><a href="/players/trent-taylor-taylotr04" title="Trent Taylor Stats">T. Taylor</a></span></td><td>30</td><td>281</td><td>9.37</td><td>16</td><td>39</td><td>0</td></tr>
<tr class="row0 right">
<td class="left"><span class="hidden-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">Victor Bolden</a></span><span class="visible-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">V. Bolden</a></span></td><td>4</td><td>23</td><td>5.75</td><td>1</td><td>16</td><td>0</td><

In [23]:
punt_returns_players = str(punt_returns_html).count('/players/') // 2
punt_returns_count = int(7 * ((str(punt_returns_html).count('/players/') // 2) + 2))
punt_returns_count

28

In [24]:
punt_returns = all[passing_count + rushing_count + receiving_count + kick_returns_count:punt_returns_count + kick_returns_count + receiving_count + rushing_count + passing_count]
punt_returns

[<td class="left"><span class="hidden-xs"><a href="/players/trent-taylor-taylotr04" title="Trent Taylor Stats">Trent Taylor</a></span><span class="visible-xs"><a href="/players/trent-taylor-taylotr04" title="Trent Taylor Stats">T. Taylor</a></span></td>,
 <td>30</td>,
 <td>281</td>,
 <td>9.37</td>,
 <td>16</td>,
 <td>39</td>,
 <td>0</td>,
 <td class="left"><span class="hidden-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">Victor Bolden</a></span><span class="visible-xs"><a href="/players/victor-bolden-boldevi01" title="Victor Bolden Stats">V. Bolden</a></span></td>,
 <td>4</td>,
 <td>23</td>,
 <td>5.75</td>,
 <td>1</td>,
 <td>16</td>,
 <td>0</td>,
 <td class="left">San Francisco</td>,
 <td>34</td>,
 <td>304</td>,
 <td>8.94</td>,
 <td>17</td>,
 <td>39</td>,
 <td>0</td>,
 <td class="left">Opponents</td>,
 <td>33</td>,
 <td>138</td>,
 <td>4.18</td>,
 <td>23</td>,
 <td>61t</td>,
 <td>1</td>]

In [25]:
passing_clean = []
rushing_clean = []
receiving_clean = []
kick_returns_clean = []
punt_returns_clean = []

for i in range(0, len(passing)):
    passing_clean.append(re.sub('<[^<]+?>', '', str(passing[i])))
    
for i in range(0, len(rushing)):
    rushing_clean.append(re.sub('<[^<]+?>', '', str(rushing[i])))
    
for i in range(0, len(receiving)):
    receiving_clean.append(re.sub('<[^<]+?>', '', str(receiving[i])))
    
for i in range(0, len(kick_returns)):
    kick_returns_clean.append(re.sub('<[^<]+?>', '', str(kick_returns[i])))

for i in range(0, len(punt_returns)):
    punt_returns_clean.append(re.sub('<[^<]+?>', '', str(punt_returns[i])))

In [26]:
passing_clean[0:10]

['Jimmy GaroppoloJ.\xa0Garoppolo',
 '178',
 '120',
 '67.4',
 '1,560',
 '8.8',
 '7',
 '3.9',
 '5',
 '2.8']

In [27]:
rushing_clean[0:10]

['Carlos HydeC.\xa0Hyde',
 '16',
 '240',
 '938',
 '3.91',
 '58.6',
 '61',
 '8',
 '44',
 'Matt BreidaM.\xa0Breida']

In [28]:
receiving_clean[0:10]

['Carlos HydeC.\xa0Hyde',
 '16',
 '59',
 '350',
 '5.93',
 '21.9',
 '18',
 '0',
 '17',
 '88']

In [29]:
kick_returns_clean[0:10]

['Victor BoldenV.\xa0Bolden',
 '19',
 '396',
 '20.84',
 '0',
 '34',
 '0',
 'Raheem MostertR.\xa0Mostert',
 '5',
 '83']

In [30]:
punt_returns_clean[0:10]

['Trent TaylorT.\xa0Taylor',
 '30',
 '281',
 '9.37',
 '16',
 '39',
 '0',
 'Victor BoldenV.\xa0Bolden',
 '4',
 '23']

In [31]:
passing_cols = ['player', 'pass_attempts', 'completions', 'compl_percent', 'passing_yards', 'yards_per_pass', 'passing_touchdowns', 'touchdown_percentage', 'interceptions', 'int_percentage', 'long', 'sack', 'loss', 'passer_rating']
rushing_cols = ['player', 'games_played', 'rush_attempts', 'rush_yards', 'yards_per_rush', 'yards_per_game', 'long', 'rush_touchdowns', 'first_downs']
receiving_cols = ['player', 'games', 'receptions', 'receiving_yards', 'yards_per_catch', 'yards_per_game', 'long', 'receiving_touchdowns', 'first_downs', 'targets', 'yards_after_catch']
kick_returns_cols = ['player', 'kick_returns', 'kick_return_yards', 'kick_return_average', 'fair_catches', 'long', 'touchdowns']
punt_returns_cols = ['player', 'punt_returns', 'punt_return_yards', 'punt_return_average', 'fair_catches', 'long', 'touchdowns']

In [32]:
passing_df = pd.DataFrame(np.array(passing_clean).reshape(passing_players + 2, len(passing_cols)), columns = passing_cols)
rushing_df = pd.DataFrame(np.array(rushing_clean).reshape(rushing_players + 2, len(rushing_cols)), columns = rushing_cols)
receiving_df = pd.DataFrame(np.array(receiving_clean).reshape(receiving_players + 2, len(receiving_cols)), columns = receiving_cols)
kick_returns_df = pd.DataFrame(np.array(kick_returns_clean).reshape(kick_returns_players + 2, len(kick_returns_cols)), columns = kick_returns_cols)
punt_returns_df = pd.DataFrame(np.array(punt_returns_clean).reshape(punt_returns_players + 2, len(punt_returns_cols)), columns = punt_returns_cols)

In [33]:
passing_df.head()

,player,pass_attempts,completions,compl_percent,passing_yards,yards_per_pass,passing_touchdowns,touchdown_percentage,interceptions,int_percentage,long,sack,loss,passer_rating
0,Jimmy GaroppoloJ. Garoppolo,178,120,67.4,"1,560",8.8,7,3.9,5,2.8,61,8,57,96.2
1,C.J. BeathardC. Beathard,224,123,54.9,"1,430",6.4,4,1.8,6,2.7,83t,19,141,69.2
2,Brian HoyerB. Hoyer,205,119,58.0,"1,245",6.1,4,2.0,4,2.0,59,16,112,74.1
3,San Francisco,607,362,59.6,"4,235",7.0,15,2.5,15,2.5,83t,43,310,78.8
4,Opponents,544,344,63.2,"3,943",7.2,27,5.0,10,1.8,72t,30,179,93.9


In [47]:
passing_df['player'].astype(str).str[0] + '. '

0    J. 
1    C. 
2    B. 
3    S. 
4    O. 
Name: player, dtype: object

In [52]:
passing_df['player'].str.split('. ',expand=True)

,0,1
0,Jimm,GaroppoloJ. Garoppolo
1,C.J,BeathardC. Beathard
2,Bria,HoyerB. Hoyer
3,Sa,Francisco
4,Opponents,None


In [34]:
rushing_df.head()

,player,games_played,rush_attempts,rush_yards,yards_per_rush,yards_per_game,long,rush_touchdowns,first_downs
0,Carlos HydeC. Hyde,16,240,938,3.91,58.6,61,8,44
1,Matt BreidaM. Breida,16,105,465,4.43,29.1,33t,2,22
2,C.J. BeathardC. Beathard,7,26,136,5.23,19.4,16,3,9
3,Marquise GoodwinM. Goodwin,16,4,44,11.00,2.8,18,0,3
4,Kyle JuszczykK. Juszczyk,14,7,31,4.43,2.2,12,0,4


In [35]:
receiving_df.head()

,player,games,receptions,receiving_yards,yards_per_catch,yards_per_game,long,receiving_touchdowns,first_downs,targets,yards_after_catch
0,Carlos HydeC. Hyde,16,59,350,5.93,21.9,18,0,17,88,322
1,Marquise GoodwinM. Goodwin,16,56,962,17.18,60.1,83t,2,46,105,198
2,Trent TaylorT. Taylor,15,43,430,10.00,28.7,33,2,27,60,190
3,George KittleG. Kittle,15,43,515,11.98,34.3,44,2,22,63,271
4,Pierre GarconP. Garcon,8,40,500,12.50,62.5,59,0,25,67,190


In [36]:
kick_returns_df.head()

,player,kick_returns,kick_return_yards,kick_return_average,fair_catches,long,touchdowns
0,Victor BoldenV. Bolden,19,396,20.84,0,34,0
1,Raheem MostertR. Mostert,5,83,16.60,0,21,0
2,Matt BreidaM. Breida,5,83,16.60,0,28,0
3,Trent TaylorT. Taylor,1,8,8.00,0,8,0
4,Ronald BlairR. Blair,1,0,0.00,0,0,0


In [37]:
punt_returns_df.head()

,player,punt_returns,punt_return_yards,punt_return_average,fair_catches,long,touchdowns
0,Trent TaylorT. Taylor,30,281,9.37,16,39,0
1,Victor BoldenV. Bolden,4,23,5.75,1,16,0
2,San Francisco,34,304,8.94,17,39,0
3,Opponents,33,138,4.18,23,61t,1
